In [1]:
import email.mime.multipart
import email.mime.nonmultipart
import email.policy
import json
import sys
import urllib.error
import urllib.request

import conllu
import pyterrier as pt

sys.path.insert(0, "..")

from src import udpipe, utils
from src.types import LAN

In [2]:
if not pt.started():
    pt.init(tqdm="notebook", logging="INFO")

Picked up _JAVA_OPTIONS: -Djava.util.prefs.userRoot=/home/dburian/.config/java
PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.



## UDPipe


In [83]:


UDPipeService = "https://lindat.mff.cuni.cz/services/udpipe/api"


def perform_request(server, method, params={}):
    if not params:
        request_headers, request_data = {}, None
    else:
        message = email.mime.multipart.MIMEMultipart(
            "form-data", policy=email.policy.HTTP
        )

        for name, value in params.items():
            payload = email.mime.nonmultipart.MIMENonMultipart("text", "plain")
            payload.add_header(
                "Content-Disposition", 'form-data; name="{}"'.format(name)
            )
            payload.add_header("Content-Transfer-Encoding", "8bit")
            payload.set_payload(value, charset="utf-8")
            message.attach(payload)

        request_data = message.as_bytes().split(b"\r\n\r\n", maxsplit=1)[1]
        request_headers = {"Content-Type": message["Content-Type"]}

    try:
        with urllib.request.urlopen(
            urllib.request.Request(
                url="{}/{}".format(server, method),
                headers=request_headers,
                data=request_data,
            )
        ) as request:
            return json.loads(request.read())
    except urllib.error.HTTPError as e:
        print(
            "An exception was raised during UDPipe 'process' REST request.\n"
            "The service returned the following error:\n"
            "  {}".format(e.fp.read().decode("utf-8")),
            file=sys.stderr,
        )
        raise
    except json.JSONDecodeError as e:
        print(
            "Cannot parse the JSON response of UDPipe 'process' REST request.\n"
            "  {}".format(e.msg),
            file=sys.stderr,
        )
        raise


def process(args: dict[str, str], text_data):
    data = {
        "input": args["input"],
        "output": args["output"],
        "data": text_data,
    }
    for option in ["model", "tokenizer", "parser", "tagger"]:
        value = args.get(option, None)
        if value is not None:
            data[option] = value

    response = perform_request(UDPipeService, "process", data)
    if "model" not in response or "result" not in response:
        raise ValueError("Cannot parse the UDPipe 'process' REST request response.")

    print(
        "UDPipe generated an output using the model '{}'.".format(response["model"]),
        file=sys.stderr,
    )
    print(
        "Please respect the model licence (CC BY-NC-SA unless stated otherwise).",
        file=sys.stderr,
    )

    return response["result"]

In [127]:
args = {
    "input": "horizontal",
    "output": "conllu",
    "model": "czech",
    "tokenizer": "",
    "tagger": "",
}
text_data = """Ahoj jak se máš já se mám dobře

babička koupila rohlíky za 20 Kč"""

return_data = process(args, text_data)

print(return_data)

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
1	Ahoj	ahoj	PART	TT-------------	_	_	_	_	_
2	jak	jak	ADV	Db-------------	PronType=Int,Rel	_	_	_	_
3	se	se	PRON	P7-X4----------	Case=Acc|PronType=Prs|Reflex=Yes|Variant=Short	_	_	_	_
4	máš	mít	VERB	VB-S---2P-AA---	Mood=Ind|Number=Sing|Person=2|Polarity=Pos|Tense=Pres|VerbForm=Fin|Voice=Act	_	_	_	_
5	já	já	PRON	PP-S1--1-------	Case=Nom|Number=Sing|Person=1|PronType=Prs	_	_	_	_
6	se	se	PRON	P7-X4----------	Case=Acc|PronType=Prs|Reflex=Yes|Variant=Short	_	_	_	_
7	mám	mít	VERB	VB-S---1P-AA---	Mood=Ind|Number=Sing|Person=1|Polarity=Pos|Tense=Pres|VerbForm=Fin|Voice=Act	_	_	_	_
8	dobře	dobře	ADV	Dg-------1A----	Degree=Pos|Polarity=Pos	_	_	_	_

# newpar
# sent_id = 2
1	babička	babička	NOUN	NNFS1-----A----	Case=Nom|Gender=Fem|Number=Sing|Polarity=Pos	_	_	_	_
2	koupila	koupit	VERB	VpQW---XR-AA---	Aspect=Perf|Gender=Fem,N

UDPipe generated an output using the model 'czech-pdt-ud-2.10-220711'.
Please respect the model licence (CC BY-NC-SA unless stated otherwise).


In [126]:

from conllu import parse

data = parse(return_data)

for sentence in data:
    for token in sentence:
        print(token["lemma"])

    print()
    print("new sentence")

ahoj
jak
se
mít
já
se
mít
dobře

new sentence
babička
koupit
rohlík
za
20
Kč

new sentence


In [3]:

ref_index = pt.IndexFactory.of("../indices/run-0_cs")
index = pt.IndexFactory.of("../run-0-udpipe-lemm_cs")

19:42:28.773 [main] INFO org.terrier.structures.FSADocumentIndex - Document index requires 319.3 KiB remaining heap is 3.9 GiB
19:42:28.899 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file into memory
19:42:28.911 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading reverse map for key docno directly from disk
19:42:28.913 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta loading data file into memory
19:42:28.932 [main] INFO org.terrier.structures.FSADocumentIndex - Document index requires 319.3 KiB remaining heap is 3.9 GiB
19:42:29.001 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file into memory
19:42:29.007 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading reverse map for key docno directly from disk
19:42:29.008 [main] INFO org.terrier.structures.BaseCompressingMetaIndex - Structure meta loadin

In [4]:
print(index.getCollectionStatistics())
print(ref_index.getCollectionStatistics())

Number of documents: 81735
Number of terms: 52522
Number of postings: 1255330
Number of fields: 1
Number of tokens: 1373064
Field names: [text]
Positions:   false

Number of documents: 81735
Number of terms: 536459
Number of postings: 13562251
Number of fields: 1
Number of tokens: 21893821
Field names: [text]
Positions:   false



In [5]:
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
docid = 1  # docids are 0-based
# NB: postings will be null if the document is empty
for posting in di.getPostings(doi.getDocumentEntry(docid)):
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    print("%s with frequency %d" % (lee.getKey(), posting.getFrequency()))

země with frequency 1
euro with frequency 1
se with frequency 1
. with frequency 1
zatím with frequency 1
komise with frequency 1
dvanáct with frequency 1
být with frequency 1
evropský with frequency 1
na with frequency 2
nadšený with frequency 1
nový with frequency 1
probíhat with frequency 1
občan with frequency 1
zvědavý with frequency 1
radovat with frequency 1
a with frequency 2
měna with frequency 1
, with frequency 1
hladce with frequency 1
většinou with frequency 1
přechod with frequency 1


In [20]:

document_paths = list(utils.paths_gen("../A1/documents_cs.lst", "../A1/documents_cs"))
documents = pt.index.treccollection2textgen(document_paths, tag_text_length=4096*2)

In [27]:
doc1 = next(documents)
doc2 = next(documents)

In [28]:
print(doc1)
print("-----")
print(doc2)

{'docno': 'LN-20020102007', 'text': '01/02/02  Ohňostroje, fronty před bankomaty a první dychtivé nákupy v nové měně. Tak 304\n  milionů Evropanů oslavilo přechod k euru, které se úderem silvestrovské\n  půlnoci stalo ve 12 zemích eurozóny oficiálním platidlem. Euro dorazilo i mimo\n  Evropu - například na francouzský ostrov Réunion vzdálený tisíce kilometrů od\n  starého kontinentu.  Nizozemci slavili v Maastrichtu  MAASTRICHT  Ohňostroje ozařovaly příchod nového roku a nizozemský ministr financí Gerrit\n  Zalm nahlas počítal: padesát, deset, deset a pět, celkem 75 eur. Právě v\n  Maastrichtu byla před deseti lety podepsána smlouva, která otevřela cestu k\n  zavedení eura. Akt, díky němuž se Maastricht zviditelnil na mapě světa a stal\n  se i centrem nizozemských oslav na počest nové měny. Vystoupení akrobatů,\n  kteří balancovali dvacet metrů nad náměstím, přihlížel i premiér Wim Kok. mží  Španělsko zažilo "eurokrádež"  MADRID  Příchod nového roku znamenal pro Španělsko hned dvě velk

In [29]:
print(udpipe.sanitize_text(doc1["text"]))
print("-----")
print(udpipe.sanitize_text(doc2["text"]))

01/02/02  Ohňostroje  fronty před bankomaty a první dychtivé nákupy v nové měně  Tak 304   milionů Evropanů oslavilo přechod k euru  které se úderem silvestrovské   půlnoci stalo ve 12 zemích eurozóny oficiálním platidlem  Euro dorazilo i mimo   Evropu - například na francouzský ostrov Réunion vzdálený tisíce kilometrů od   starého kontinentu   Nizozemci slavili v Maastrichtu  MAASTRICHT  Ohňostroje ozařovaly příchod nového roku a nizozemský ministr financí Gerrit   Zalm nahlas počítal  padesát  deset  deset a pět  celkem 75 eur  Právě v   Maastrichtu byla před deseti lety podepsána smlouva  která otevřela cestu k   zavedení eura  Akt  díky němuž se Maastricht zviditelnil na mapě světa a stal   se i centrem nizozemských oslav na počest nové měny  Vystoupení akrobatů    kteří balancovali dvacet metrů nad náměstím  přihlížel i premiér Wim Kok  mží  Španělsko zažilo  eurokrádež   MADRID  Příchod nového roku znamenal pro Španělsko hned dvě velké události spojené se   členstvím země v Evrop

In [30]:

lemmatized_docs = udpipe.lemmatize_doc(
    doc1, lan=LAN.CS, udpipe_service=udpipe.UD_PIPE_SERVICE
)

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = 01/02/02 Ohňostroje, fronty před bankomaty a první dychtivé nákupy v nové měně.
1	01	01	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpaceAfter=No
2	/	/	PUNCT	Z:-------------	_	_	_	_	SpaceAfter=No
3	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpaceAfter=No
4	/	/	PUNCT	Z:-------------	_	_	_	_	SpaceAfter=No
5	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpacesAfter=\s\s
6	Ohňostroje	ohňostroj	NOUN	NNIP1-----A----	Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur|Polarity=Pos	_	_	_	SpaceAfter=No
7	,	,	PUNCT	Z:-------------	_	_	_	_	_
8	fronty	fronta	NOUN	NNFP1-----A----	Case=Nom|Gender=Fem|Number=Plur|Polarity=Pos	_	_	_	_
9	před	před	ADP	RR--7----------	AdpType=Prep|Case=Ins	_	_	_	_
10	bankomaty	bankomat	NOUN	NNIP7-----A----	Animacy=Inan|Case=Ins|Gender=Masc|Number=Plur|Pol

In [31]:
print(lemmatized_docs)

{'docno': 'LN-20020102007', 'text': '01 / 02 / 02 ohňostroj , fronta před bankomat a první dychtivý nákup v nový měna .tak 304 milión Evropan oslavit přechod k euro , který se úder silvestrovský půlnoc stát v 12 země eurozóna oficiální platidlo .Euro dorazit i mimo Evropa - například na francouzský ostrov Réunion vzdálený tisíc kilometr od starý kontinent .Nizozemec slavit v Maastricht Maastrichtohňostroj ozařovat příchod nový rok a nizozemský ministr finance Gerrit Zalm nahlas počítat : padesát , deset , deset a pět , celkem 75 eura .právě v Maastricht být před deset rok podepsaný smlouva , který otevřít cesta k zavedení euro .akt , díky jenž se maastricht zviditelnit na mapa svět a stát se i centrum nizozemský oslava na počest nový měna .vystoupení akrobat , který balancovat dvacet metr nad náměstí , přihlížet i premiér Wim Kok .mží Španělsko zažít " eurokrádež " Madrid příchod nový rok znamenat pro Španělsko hned dva velký událost spojený s členství země v evropský unie .kromě spole

In [20]:
lemmatized_doc_text = udpipe._lemmatize_text(
    [doc1["text"], doc2["text"]],
    lan=LAN.CS,
    udpipe_service=udpipe.UD_PIPE_SERVICE,
)

01 / 02 / 02

v doba kdy eurobankomat už několik hodina vydávat eurobankovka zavolat přítel z New York popřát já všechen dobrý zeptat se na rodina a pak prohodit slyšet být že zavádět evropský dolar


# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = 01 / 02 / 02
1	01	01	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	_
2	/	/	PUNCT	Z:-------------	_	_	_	_	_
3	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	_
4	/	/	PUNCT	Z:-------------	_	_	_	_	_
5	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpacesAfter=\r\n\r\n

# newpar
# sent_id = 2
# text = v doba kdy eurobankomat už několik hodina vydávat eurobankovka zavolat přítel z New York popřát já všechen dobrý zeptat se na rodina a pak prohodit slyšet být že zavádět evropský dolar
1	v	v	ADP	RR--4----------	AdpType=Prep|Case=Acc	_	_	_	_
2	doba	doba	NOUN	NNFS4-----A----	Case=Acc

In [23]:
print(lemmatized_doc_text)

['01 / 02 / 02', 'v doba kdy eurobankomat už několik hodina vydávat eurobankovka zavolat přítel z New York popřát já všechen dobrý zeptat se na rodina a pak prohodit slyšet být že zavádět evropský dolar']


In [26]:
def _lemmatize_list(texts: list[str], lan: LAN, udpipe_service) -> list[str]:
    text_data = ""
    for text in texts:
        text_data += udpipe.sanitize_text(text)
        text_data += "\n\n"

    print(text_data)

    params = {
        "input": "",
        "output": "conllu",
        "model": "czech" if lan == LAN.CS else "english",
        "tokenizer": "",
        "tagger": "",
        "data": text_data.rstrip(),
    }
    response = udpipe._perform_request(params=params, server=udpipe_service)
    return response["result"]

In [37]:
lemmatized_doc_text = _lemmatize_list(
    [doc1["text"], doc2["text"]],
    lan=LAN.CS,
    udpipe_service=udpipe.UD_PIPE_SERVICE,
)

01/02/02  Ohňostroje  fronty před bankomaty a první dychtivé nákupy v nové měně  Tak 304 milionů Evropanů oslavilo přechod k euru  které se úderem silvestrovské půlnoci stalo ve 12 zemích eurozóny oficiálním platidlem  Euro dorazilo i mimo Evropu - například na francouzský ostrov Réunion vzdálený tisíce kilometrů od starého kontinentu   Nizozemci slavili v Maastrichtu  MAASTRICHT  Ohňostroje ozařovaly příchod nového roku a nizozemský ministr financí Gerrit Zalm nahlas počítal  padesát  deset  deset a pět  celkem 75 eur  Právě v Maastrichtu byla před deseti lety podepsána smlouva  která otevřela cestu k zavedení eura  Akt  díky němuž se Maastricht zviditelnil na mapě světa a stal se i centrem nizozemských oslav na počest nové měny  Vystoupení akrobatů  kteří balancovali dvacet metrů nad náměstím  přihlížel i premiér Wim Kok  mží  Španělsko zažilo  eurokrádež   MADRID  Příchod nového roku znamenal pro Španělsko hned dvě velké události spojené se členstvím země v Evropské unii  Kromě spol

In [40]:
print(lemmatized_doc_text)
conllu_parsed = conllu.parse(lemmatized_doc_text)

# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = czech-pdt-ud-2.10-220711
# udpipe_model_licence = CC BY-NC-SA
# newdoc
# newpar
# sent_id = 1
# text = 01/02/02
1	01	01	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpaceAfter=No
2	/	/	PUNCT	Z:-------------	_	_	_	_	SpaceAfter=No
3	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpaceAfter=No
4	/	/	PUNCT	Z:-------------	_	_	_	_	SpaceAfter=No
5	02	02	NUM	C=-------------	NumForm=Digit|NumType=Card	_	_	_	SpacesAfter=\s\s

# sent_id = 2
# text = Ohňostroje fronty před bankomaty a první dychtivé nákupy v nové měně
1	Ohňostroje	ohňostroj	NOUN	NNIP1-----A----	Animacy=Inan|Case=Nom|Gender=Masc|Number=Plur|Polarity=Pos	_	_	_	SpacesAfter=\s\s
2	fronty	fronta	NOUN	NNFS2-----A----	Case=Gen|Gender=Fem|Number=Sing|Polarity=Pos	_	_	_	_
3	před	před	ADP	RR--7----------	AdpType=Prep|Case=Ins	_	_	_	_
4	bankomaty	bankomat	NOUN	NNIP7-----A----	Animacy=Inan|Case=Ins|Gender=Masc|Number=Plur|Polarity=Pos	_	_	

In [ ]:

for sentence in conllu_parsed: